In [38]:
import numpy as np
from nltk.corpus import wordnet as wn
from stanfordcorenlp import StanfordCoreNLP
import re
import bisect
from collections import defaultdict
import ast
import os
from gutenberg.cleanup import strip_headers
from nltk.tokenize import sent_tokenize
from bs4 import BeautifulSoup
import math
import gensim
import pickle
from scipy import spatial
from nltk.tree import *
import nltk.corpus
import nltk.tokenize.punkt
import nltk.stem.snowball
import string
from multiprocessing import Pool
from nltk.draw.tree import TreeView
from fuzzywuzzy import fuzz
from multiprocessing import Pool

Exploring bigram and trigram based discarding of sentences

In [200]:
public='/home/users2/mehrotsh/scripts/packages/stanford-corenlp-full-2018-02-27/'
personal='/home/samarth/stanford-corenlp-full-2018-02-27/'

In [201]:
nlp = StanfordCoreNLP(public)

In [2]:
potential="../data/potential/"
booksList=os.listdir(potential)

In [3]:
books=dict()
for file in booksList:
    print(file)
    candidate=open(potential+file)
    rawtext=candidate.read()
    candidate.close()
    rawtext = strip_headers(rawtext).strip()
    candidate=rawtext.replace('\n',' ')
    candidate=rawtext.replace(':','. ')
    candidate=sent_tokenize(candidate)
    candidate = list(filter(lambda x: len(x)>5, candidate))
    books[file]=candidate

Joshua
hosea
exodus
esther
amos
Ezra
genesis
kings-3
leviticus
samuel-2
Proverbs
Deuteronomy
end
isaiah
chronicle-2
daniel
joel
Numbers
nehemiah
jeremiah-2
Psalms
jeremiah-1
Judges
samuel-1
chronicle-1
preacher
Ruth
kings-4
job


In [4]:
len(books['isaiah'])

2089

In [5]:
tokenizer = nltk.tokenize.TreebankWordTokenizer()

In [11]:
test="./new/newTestament.txt"
testB=open(test)
raw=testB.read()
text = strip_headers(raw).strip()
text=text.replace('\n',' ')
text=text.replace(':','. ')
words=[token.lower().strip(string.punctuation) for token in tokenizer.tokenize(text)]
text=sent_tokenize(text)
text = list(filter(lambda x: len(x)>5, text))
testB.close()

In [12]:
len(words)

217551

In [13]:
bigrams=nltk.bigrams(words)

In [14]:
bigramsFreq=nltk.FreqDist(bigrams)

In [21]:
len(bigramsFreq)

57118

In [17]:
bigramsFreq.most_common(10)

[(('', 'and'), 5409),
 (('of', 'the'), 1774),
 (('him', ''), 958),
 (('', 'that'), 906),
 (('and', 'the'), 897),
 (('in', 'the'), 881),
 (('and', 'he'), 667),
 (('', 'but'), 665),
 (('of', 'god'), 605),
 (('them', ''), 585)]

In [18]:
trigrams=nltk.trigrams(words)

In [19]:
trigramsFreq=nltk.FreqDist(trigrams)

In [25]:
mostCommonBigrams=[]
for i in range(10):
    mostCommonBigrams.append(bigramsFreq.most_common(10)[i][0])

In [26]:
mostCommonBigrams

[('', 'and'),
 ('of', 'the'),
 ('him', ''),
 ('', 'that'),
 ('and', 'the'),
 ('in', 'the'),
 ('and', 'he'),
 ('', 'but'),
 ('of', 'god'),
 ('them', '')]

In [23]:
mostCommonTrigrams=[]
for i in range(10):
    mostCommonTrigrams.append(trigramsFreq.most_common(10)[i][0])

In [24]:
mostCommonTrigrams

[('', 'and', 'the'),
 ('unto', 'him', ''),
 ('unto', 'them', ''),
 ('', 'saying', ''),
 ('the', 'son', 'of'),
 ('him', '', 'and'),
 ('unto', 'you', ''),
 ('of', 'god', ''),
 ('said', 'unto', 'them'),
 ('said', 'unto', 'him')]

In [30]:
for sent in text:
    sent_words=[token.lower().strip(string.punctuation) for token in tokenizer.tokenize(sent)]
    sent_bigrams=list(nltk.bigrams(sent_words))
    sent_trigrams=list(nltk.trigrams(sent_words))
    bigramFreq=0
    for bi in sent_bigrams:
        if bi in mostCommonBigrams:
            bigramFreq=bigramFreq+1
    trigramFreq=0
    for tri in sent_trigrams:
        if tri in mostCommonTrigrams:
            trigramFreq=trigramFreq+1
    if len(sent_bigrams)==0 or len(sent_trigrams)==0:
        print(0,0)
    else:
        print(bigramFreq/len(sent_bigrams),trigramFreq/len(sent_trigrams))

0.06666666666666667 0.0
0.05263157894736842 0.1111111111111111
0.10526315789473684 0.0
0.09523809523809523 0.0
0.11764705882352941 0.0
0.09523809523809523 0.0
0.04 0.0
0.11764705882352941 0.0
0.11764705882352941 0.0
0.11764705882352941 0.0
0.11764705882352941 0.0
0.0 0.0
0.05263157894736842 0.0
0.11764705882352941 0.0
0.11764705882352941 0.0
0.11764705882352941 0.0
0.0 0.0
0.05263157894736842 0.0
0.0 0.0
0.041666666666666664 0.0
0.03571428571428571 0.0
0.024390243902439025 0.025
0.08333333333333333 0.0
0.0625 0.0
0.0 0.0
0.08 0.041666666666666664
0.06060606060606061 0.0
0.125 0.043478260869565216
0.0 0.0
0.16666666666666666 0.0
0.03225806451612903 0.0
0.07692307692307693 0.0
0.1875 0.0
0.10526315789473684 0.0
0.038461538461538464 0.0
0.09090909090909091 0.2
0.0 0.0
0.05 0.0
0.06666666666666667 0.0
0.0 0.0
0.125 0.0
0.075 0.0
0.0 0.0
0.1111111111111111 0.0
0.05 0.0
0.04 0.0
0.08333333333333333 0.02127659574468085
0.1 0.0
0.05 0.0
0.0 0.0
0.04 0.041666666666666664
0.08196721311475409 0.0

0.0 0.0
0.0 0.0
0.0625 0.0
0.06666666666666667 0.0
0.0 0.0
0.0 0.058823529411764705
0.0 0.0
0.02631578947368421 0.02702702702702703
0.09523809523809523 0.0
0.16 0.041666666666666664
0.0 0.0
0.06666666666666667 0.07142857142857142
0.0 0.0
0.0 0.0
0.047619047619047616 0.05
0.1111111111111111 0.0
0.11538461538461539 0.04
0.14285714285714285 0.05
0.047619047619047616 0.0
0.14285714285714285 0.1
0.1111111111111111 0.038461538461538464
0.2 0.0
0.0 0.0
0.038461538461538464 0.0
0.0625 0.06451612903225806
0.0 0.0
0.08333333333333333 0.043478260869565216
0.0 0.041666666666666664
0.06451612903225806 0.0
0.25 0.09090909090909091
0.13333333333333333 0.10344827586206896
0.10344827586206896 0.07142857142857142
0.047619047619047616 0.0
0.0 0.0
0.09090909090909091 0.0
0.0 0.0
0.08695652173913043 0.045454545454545456
0.09523809523809523 0.05
0.0 0.0
0.037037037037037035 0.038461538461538464
0.0 0.0
0.08 0.041666666666666664
0.045454545454545456 0.09523809523809523
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.14285

0.19047619047619047 0.15
0.0 0.0
0.13513513513513514 0.0
0.16666666666666666 0.18181818181818182
0.0 0.0
0.0 0.0
0.05405405405405406 0.0
0.0 0.0
0.06666666666666667 0.0
0.0 0.0
0.0 0.0
0.1935483870967742 0.06666666666666667
0.08108108108108109 0.0
0.09090909090909091 0.09523809523809523
0.0 0.0
0.038461538461538464 0.0
0.0 0.0
0.16666666666666666 0.18181818181818182
0.0 0.0
0.0 0.0
0.0 0.0
0.0625 0.0
0.08333333333333333 0.043478260869565216
0.125 0.043478260869565216
0.038461538461538464 0.0
0.047619047619047616 0.0
0.15384615384615385 0.08333333333333333
0.0 0.0
0.06451612903225806 0.0
0.07894736842105263 0.0
0.05 0.05263157894736842
0.1 0.1111111111111111
0.0 0.0
0.08 0.041666666666666664
0.05 0.0
0.0 0.0
0.08333333333333333 0.043478260869565216
0.1875 0.0
0.16666666666666666 0.11764705882352941
0.15 0.0
0.18181818181818182 0.0
0.16666666666666666 0.18181818181818182
0.0 0.0
0.14285714285714285 0.0
0.10526315789473684 0.0
0.07407407407407407 0.0
0.09523809523809523 0.0
0.172413793103

0.03225806451612903 0.06666666666666667
0.125 0.13333333333333333
0.0967741935483871 0.1
0.07692307692307693 0.16666666666666666
0.1 0.0
0.0 0.0
0.1111111111111111 0.25
0.0967741935483871 0.06666666666666667
0.03225806451612903 0.0
0.0 0.0
0.15789473684210525 0.05405405405405406
0.0 0.0
0.0 0.0
0.0 0.0
0.07407407407407407 0.038461538461538464
0.0 0.0
0.0 0.03333333333333333
0.03125 0.0
0.0 0.0
0.0 0.0
0.08333333333333333 0.0
0.047619047619047616 0.05
0.13333333333333333 0.0
0.05555555555555555 0.11764705882352941
0.13333333333333333 0.14285714285714285
0.058823529411764705 0.125
0.15384615384615385 0.0
0.0 0.0
0.06666666666666667 0.07142857142857142
0.10714285714285714 0.037037037037037035
0.0 0.08333333333333333
0.11764705882352941 0.0625
0.10714285714285714 0.0
0.05263157894736842 0.1111111111111111
0.0 0.1
0.0 0.0
0.23809523809523808 0.05
0.1111111111111111 0.0
0.13333333333333333 0.0
0.10344827586206896 0.03571428571428571
0.16666666666666666 0.0
0.25 0.0
0.09090909090909091 0.0
0.

0.025 0.0
0.16666666666666666 0.0
0.07692307692307693 0.08333333333333333
0.08333333333333333 0.0
0.2222222222222222 0.125
0.038461538461538464 0.0
0.05263157894736842 0.05555555555555555
0.041666666666666664 0.0
0.043478260869565216 0.0
0.11538461538461539 0.04
0.08 0.0
0.0 0.0
0.07692307692307693 0.0
0.08333333333333333 0.0
0.0 0.0
0.08333333333333333 0.0
0.13333333333333333 0.07142857142857142
0.07142857142857142 0.0
0.09090909090909091 0.0
0.0 0.0
0.14814814814814814 0.038461538461538464
0.05555555555555555 0.11764705882352941
0.0 0.0
0.0 0.0
0.0 0.0
0.05 0.0
0.16666666666666666 0.0
0.041666666666666664 0.0
0.08333333333333333 0.0
0.0625 0.0
0.07407407407407407 0.0
0.05555555555555555 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.05555555555555555 0.0
0.0 0.0
0.05 0.0
0.058823529411764705 0.0
0.1111111111111111 0.058823529411764705
0.0 0.0
0.058823529411764705 0.0
0.0625 0.0
0.05263157894736842 0.0
0.043478260869565216 0.0
0.06451612903225806 0.0
0.05882352941176470

...

0.125 0.0
0.05263157894736842 0.0
0.0 0.0
0.0 0.0
0.05263157894736842 0.0
0.03571428571428571 0.0
0.043478260869565216 0.0
0.125 0.0
0.0 0.0
0.0 0.0
0.043478260869565216 0.0
0.2 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.06896551724137931 0.0
0.09375 0.0
0.06666666666666667 0.0
0.045454545454545456 0.0
0.0 0.0
0.15384615384615385 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.03571428571428571 0.0
0.0 0.0
0.0 0.0
0.047619047619047616 0.0
0.10526315789473684 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.20833333333333334 0.0
0.05555555555555555 0.0
0.05 0.0
0.1111111111111111 0.125
0.058823529411764705 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.058823529411764705 0.0
0.0 0.0
0.1 0.0
0.0 0.0
0.04 0.0
0.08 0.0
0.0 0.0
0.0 0.0
0.058823529411764705 0.0625
0.07407407407407407 0.038461538461538464
0.029411764705882353 0.0
0.0 0.0
0.10526315789473684 0.0
0.0625 0.0
0.06666666666666667 0.0
0.03125 0.0
0.05555555555555555 0.02857142857142857
0.08108108108108109 0.027777777777777776
0.0 0.0
0.0 0.0
0.085714

0.21052631578947367 0.05555555555555555
0.0 0.0
0.0 0.0
0.0 0.0
0.09090909090909091 0.0
0.0 0.0
0.08333333333333333 0.043478260869565216
0.18181818181818182 0.0
0.1 0.0
0.0 0.0
0.08333333333333333 0.0
0.0 0.0
0.045454545454545456 0.0
0.0 0.0
0.0 0.0
0.03225806451612903 0.0
0.0 0.0
0.08 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.08333333333333333 0.0
0.05263157894736842 0.0
0.0 0.0
0.12195121951219512 0.025
0.0 0.0
0.07692307692307693 0.05263157894736842
0.09523809523809523 0.0
0.10526315789473684 0.0
0.16 0.041666666666666664
0.045454545454545456 0.0
0.13333333333333333 0.0
0.1111111111111111 0.0
0.034482758620689655 0.0
0.08333333333333333 0.0
0.06060606060606061 0.0
0.041666666666666664 0.0
0.08 0.041666666666666664
0.041666666666666664 0.0
0.0 0.0
0.05 0.0
0.05555555555555555 0.0
0.06060606060606061 0.03125
0.0 0.0
0.0 0.0
0.0 0.0
0.1 0.0
0.04 0.0
0.04 0.0
0.0 0.0
0.08 0.0
0.1111111111111111 0.0
0.02631578947368421 0.0
0.0967741935483871 0.0
0.08108108108108109 0.027777777777777776
0.0 0.0
0.0 0

0.045454545454545456 0.0
0.0 0.0
0.06666666666666667 0.0
0.05405405405405406 0.0
0.3 0.1111111111111111
0.14285714285714285 0.0
0.16666666666666666 0.0
0.05 0.0
0.0 0.0
0.02564102564102564 0.0
0.043478260869565216 0.0
0.047619047619047616 0.0
0.08163265306122448 0.0
0.05714285714285714 0.0
0.06666666666666667 0.034482758620689655
0.045454545454545456 0.0
0.03225806451612903 0.0
0.07407407407407407 0.0
0.0 0.0
0.08333333333333333 0.0
0.1590909090909091 0.023255813953488372
0.0 0.0
0.0 0.0
0.12121212121212122 0.03125
0.0625 0.0
0.125 0.043478260869565216
0.06666666666666667 0.0
0.08333333333333333 0.0
0.058823529411764705 0.0
0.06666666666666667 0.0
0.06666666666666667 0.034482758620689655
0.06666666666666667 0.0
0.06060606060606061 0.0
0.0 0.0
0.0625 0.0
0.0625 0.0
0.0 0.0
0.06666666666666667 0.034482758620689655
0.06451612903225806 0.0
0.0 0.0
0.0 0.0
0.14285714285714285 0.0
0.1 0.0
0.04 0.041666666666666664
0.0 0.0
0.06666666666666667 0.0
0.0 0.0
0.037037037037037035 0.0
0.0625 0.0
0.

0.0 0.0
0.0 0.0
0.0 0.0
0.038461538461538464 0.0
0.0 0.0
0.06666666666666667 0.0
0.0 0.0
0.0 0.0
0.08333333333333333 0.0
0.05 0.0
0.0 0.0
0.038461538461538464 0.04
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.05263157894736842 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.03571428571428571 0.0
0.07692307692307693 0.08333333333333333
0.03225806451612903 0.0
0.0 0.0
0.0 0.0
0.05 0.0
0.0 0.0
0.038461538461538464 0.0
0.0 0.0
0.0 0.0
0.07547169811320754 0.038461538461538464
0.0 0.0
0.0 0.0
0.06666666666666667 0.0
0.0 0.0
0.03571428571428571 0.0
0.029411764705882353 0.0
0.0 0.0
0.08333333333333333 0.043478260869565216
0.15789473684210525 0.0
0.0 0.0
0.06666666666666667 0.0
0.08333333333333333 0.0
0.0 0.0
0.0 0.0
0.03333333333333333 0.034482758620689655
0.0 0.0
0.0 0.0
0.08695652173913043 0.0
0.07692307692307693 0.0
0.1 0.0
0.08333333333333333 0.0
0.0625 0.0
0.05555555555555555 0.0
0.06060606060606061 0.0
0.0 0.0
0.03125 0.0
0.08108108108108109 0.0
0.14634146341463414 0.0
0.0 0.0
0.0 0.0
0.07142857142857142 0.0
0.0 0.0

0.2413793103448276 0.07142857142857142
0.05555555555555555 0.0
0.0 0.0
0.19047619047619047 0.05
0.18181818181818182 0.0
0.0 0.0
0.09375 0.0
0.1276595744680851 0.021739130434782608
0.07692307692307693 0.0
0.07142857142857142 0.0
0.14285714285714285 0.0
0.0 0.0
0.0 0.0
0.08571428571428572 0.029411764705882353
0.037037037037037035 0.0
0.038461538461538464 0.0
0.1 0.0
0.0 0.0
0.0 0.0
0.07142857142857142 0.0
0.08695652173913043 0.0
0.09375 0.0
0.16666666666666666 0.08695652173913043
0.15 0.05263157894736842
0.1 0.0
0.09090909090909091 0.0
0.08108108108108109 0.0
0.125 0.0
0.1 0.0
0.13636363636363635 0.0
0.0 0.0
0.075 0.0
0.10526315789473684 0.0
0.09302325581395349 0.023809523809523808
0.1 0.0
0.14634146341463414 0.0
0.0 0.0
0.08 0.0
0.06666666666666667 0.0
0.034482758620689655 0.0
0.038461538461538464 0.0
0.058823529411764705 0.0
0.029411764705882353 0.0
0.021739130434782608 0.0
0.05555555555555555 0.0
0.043478260869565216 0.0
0.06896551724137931 0.0
0.075 0.05128205128205128
0.21875 0.0322

In [36]:
text[24:27]

['for he shall save his people from their sins.',
 '22 Now all this was done, that it might be fulfilled which was spoken of the Lord by the prophet, saying, 1.',
 '23 Behold, a virgin shall be with child, and shall bring forth a son, and they shall call his name Emmanuel, which being interpreted is, God with us.']

In [35]:
for sent in text[24:27]:
    sent_words=[token.lower().strip(string.punctuation) for token in tokenizer.tokenize(sent)]
    sent_bigrams=list(nltk.bigrams(sent_words))
    sent_trigrams=list(nltk.trigrams(sent_words))
    bigramFreq=0
    for bi in sent_bigrams:
        if bi in mostCommonBigrams:
            bigramFreq=bigramFreq+1
    trigramFreq=0
    for tri in sent_trigrams:
        if tri in mostCommonTrigrams:
            trigramFreq=trigramFreq+1
    if len(sent_bigrams)==0 or len(sent_trigrams)==0:
        print(0,0)
    else:
        print(bigramFreq/len(sent_bigrams),trigramFreq/len(sent_trigrams))

0.0 0.0
0.08 0.041666666666666664
0.06060606060606061 0.0


In [38]:
text.index('18 In Rama was there a voice heard, lamentation, and weeping, and great mourning, Rachel weeping for her children, and would not be comforted, because they are not.')

53

In [39]:
for sent in text[51:54]:
    sent_words=[token.lower().strip(string.punctuation) for token in tokenizer.tokenize(sent)]
    sent_bigrams=list(nltk.bigrams(sent_words))
    sent_trigrams=list(nltk.trigrams(sent_words))
    bigramFreq=0
    for bi in sent_bigrams:
        if bi in mostCommonBigrams:
            bigramFreq=bigramFreq+1
    trigramFreq=0
    for tri in sent_trigrams:
        if tri in mostCommonTrigrams:
            trigramFreq=trigramFreq+1
    if len(sent_bigrams)==0 or len(sent_trigrams)==0:
        print(0,0)
    else:
        print(bigramFreq/len(sent_bigrams),trigramFreq/len(sent_trigrams))

0.08196721311475409 0.0
0.0 0.06666666666666667
0.08823529411764706 0.0


Similarity between sentences using wordnet

In [48]:
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet 

In [49]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

In [50]:
def tagged_to_synset(word, tag):
    wn_tag = get_wordnet_pos(tag)
    if wn_tag is None:
        return None
 
    try:
        return wn.synsets(word, wn_tag)[0]
    except:
        return None

In [75]:
def sentenceSimilarity(sent1,sent2):
    
#     sent1words=word_tokenize(sent1)
#     sent2words=word_tokenize(sent2)
    sent1words=sent1
    sent2words=sent2
    
    sent1tags=pos_tag(sent1words)
    sent2tags=pos_tag(sent2words)
    
    
    sent1tags=[t[1] for t in sent1tags]
    sent2tags=[t[1] for t in sent2tags]
    
    sent1synset=[]
    for i in range(len(sent1words)):
        sent1synset.append(tagged_to_synset(sent1words[i],sent1tags[i]))
        
    
    sent2synset=[]
    for i in range(len(sent2words)):
        sent2synset.append(tagged_to_synset(sent2words[i],sent2tags[i]))
        
        
    sent1synset = [ss for ss in sent1synset if ss is not None]
    sent2synset = [ss for ss in sent2synset if ss is not None]
 
    score, count = 0.0, 0
 
    # For each word in the first sentence
    for synset in sent1synset:
        # Get the similarity value of the most similar word in the other sentence
        try:
            best_score = max([synset.path_similarity(ss) for ss in sent2synset])
        except TypeError:
            best_score=None
        # Check that the similarity could have been computed
        if best_score is not None:
            score += best_score
            count += 1
    if count==0:
        return 0
    else:
        score /= count
    return score
 
        
        
        
        
        
    

In [8]:
sent1='He that overcometh shall inherit all things; and I will be his god and he shall be my son'
sent2='I will be his father and he shall be my son'

In [64]:
sentenceSimilarity(text[53],text[26])

0.7166666666666667

In [9]:
sentenceSimilarity(sent1,sent2)

0.75

In [50]:
w=word_tokenize(text[0])

In [52]:
pos_tag(w)[1]

[('The', 'DT'),
 ('New', 'NNP'),
 ('Testament', 'NNP'),
 ('of', 'IN'),
 ('the', 'DT'),
 ('King', 'NNP'),
 ('James', 'NNP'),
 ('Bible', 'NNP'),
 ('The', 'DT'),
 ('Gospel', 'NNP'),
 ('According', 'VBG'),
 ('to', 'TO'),
 ('Saint', 'NNP'),
 ('Matthew', 'NNP'),
 ('1', 'CD'),
 ('.', '.')]

In [55]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/users2/mehrotsh/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

Exploring results and filtering out false positives

In [205]:
import numpy as np
from nltk.corpus import wordnet as wn
from stanfordcorenlp import StanfordCoreNLP
import re
import bisect
from collections import defaultdict
import ast
import os
from gutenberg.cleanup import strip_headers
from nltk.tokenize import sent_tokenize
from bs4 import BeautifulSoup
import math
import gensim
import pickle
from scipy import spatial
from nltk.tree import *
import nltk.corpus
import nltk.tokenize.punkt
import nltk.stem.snowball
import string
from multiprocessing import Pool
from nltk.draw.tree import TreeView
from fuzzywuzzy import fuzz
from multiprocessing import Pool

In [206]:
test="../data/new/newTestament.txt"
testB=open(test)
raw=testB.read()
text = strip_headers(raw).strip()
text=text.replace('\n',' ')
text=text.replace(':','. ')
text=sent_tokenize(text)
text = list(filter(lambda x: len(x)>5, text))

In [207]:
pickle_off = open("..//output/testPackage/scoreTuples.pickle","rb")
scoreTuples = pickle.load(pickle_off)

In [208]:
len(scoreTuples)

5980150

In [209]:
pickle_off = open("../output/testPackage/reducedBooks.pickle","rb")
reducedBooks = pickle.load(pickle_off)

In [210]:
scoreTuples.sort(key=lambda tup: tup[0])

In [15]:
for t in scoreTuples[0:500]:
    print('Original Sentence: ',text[t[0]])
    print('Similar Sentence is from: ',t[1])
    print('Syntactic Score: ',t[3])
    print('Semantic Score: ',t[4])
    print('Average: ',t[5])
    print(reducedBooks[t[1]][t[2]])
    print('\n\n')

Original Sentence:  20 But while he thought on these things, behold, the angel of the LORD appeared unto him in a dream, saying, Joseph, thou son of David, fear not to take unto thee Mary thy wife.
Similar Sentence is from:  genesis
Syntactic Score:  0.922024047174
Semantic Score:  0.916171371937
Average:  0.919097709555
1 And the LORD said unto Noah, Come thou and all thy house into the
ark; for thee have I seen righteous before me in this generation.



Original Sentence:  40 And the King shall answer and say unto them, Verily I say unto you, Inasmuch as ye have done it unto one of the least of these my brethren, ye have done it unto me.
Similar Sentence is from:  samuel-2
Syntactic Score:  0.908029911541
Semantic Score:  0.919179081917
Average:  0.913604496729
12 Go and say unto
David, Thus saith the LORD, I offer thee three things; choose thee one
of them, that I may do it unto thee.



Original Sentence:  31 When the Son of man shall come in his glory, and all the holy angels with

Average:  0.850272141365
27 And the Levite that is within thy
gates; thou shalt not forsake him; for he hath no part nor inheritance
with thee.



Original Sentence:  15 And Jesus answering said unto him, Suffer it to be so now.
Similar Sentence is from:  genesis
Syntactic Score:  0.870547678688
Semantic Score:  0.828246295452
Average:  0.84939698707
14 And Joseph said unto them, That is it that I spake unto you,
saying, Ye are spies.



Original Sentence:  26 Verily I say unto thee, Thou shalt by no means come out thence, till thou hast paid the uttermost farthing.
Similar Sentence is from:  Deuteronomy
Syntactic Score:  0.85158224991
Semantic Score:  0.847199082375
Average:  0.849390666142
29 When the LORD thy God shall cut off the nations from before
thee, whither thou goest to possess them, and thou succeedest them,
and dwellest in their land; 12.



Original Sentence:  26 Verily I say unto thee, Thou shalt by no means come out thence, till thou hast paid the uttermost farthing.
Si

shall not prevail.



Original Sentence:  this same Jesus, which is taken up from you into heaven, shall so come in like manner as ye have seen him go into heaven.
Similar Sentence is from:  Numbers
Syntactic Score:  0.819179100208
Semantic Score:  0.855992436409
Average:  0.837585768309
for he shall not enter into the land which I have given
unto the children of Israel, because ye rebelled against my word at
the water of Meribah.



Original Sentence:  26 Verily I say unto thee, Thou shalt by no means come out thence, till thou hast paid the uttermost farthing.
Similar Sentence is from:  jeremiah-2
Syntactic Score:  0.841558307013
Semantic Score:  0.83350032568
Average:  0.837529316346
5 Thus saith the Lord GOD unto these bones; Behold, I will cause
breath to enter into you, and ye shall live.



Original Sentence:  19 He shall not strive, nor cry; neither shall any man hear his voice in the streets.
Similar Sentence is from:  amos
Syntactic Score:  0.806843794716
Semantic Score:  0.8

Syntactic Score:  0.862301695211
Semantic Score:  0.800086438656
Average:  0.831194066934
that that dieth, let it die;
and that that is to be cut off, let it be cut off; and let the rest
eat every one the flesh of another.



Original Sentence:  20 A bruised reed shall he not break, and smoking flax shall he not quench, till he send forth judgment unto victory.
Similar Sentence is from:  Deuteronomy
Syntactic Score:  0.857419768695
Semantic Score:  0.804956674576
Average:  0.831188221635
29 Then
the man that lay with her shall give unto the damsel's father fifty
shekels of silver, and she shall be his wife; because he hath humbled
her, he may not put her away all his days.



Original Sentence:  20 A bruised reed shall he not break, and smoking flax shall he not quench, till he send forth judgment unto victory.
Similar Sentence is from:  Deuteronomy
Syntactic Score:  0.849059763612
Semantic Score:  0.813251912594
Average:  0.831155838103
16 And it shall be, if he say unto thee, I will 

In [16]:
sentNumbers=list()

In [17]:
for t in scoreTuples[0:1000]:
    sentNumbers.append(t[0])

In [18]:
from collections import Counter

In [19]:
counts = Counter(sentNumbers)

In [20]:
print(counts)

Counter({161: 222, 172: 97, 5346: 80, 162: 69, 2613: 63, 517: 45, 163: 39, 1199: 38, 5351: 28, 2592: 27, 2584: 25, 2586: 21, 2582: 19, 1: 15, 5870: 15, 5872: 15, 1196: 13, 5350: 13, 2931: 13, 518: 10, 2614: 10, 2926: 9, 5908: 8, 2595: 7, 6750: 6, 2932: 6, 5916: 5, 5433: 4, 173: 3, 31: 3, 2619: 3, 2585: 2, 11: 2, 5455: 2, 2581: 2, 2: 2, 2621: 2, 32: 2, 5520: 2, 2594: 2, 21: 1, 1266: 1, 1250: 1, 5474: 1, 5716: 1, 5954: 1, 1240: 1, 3058: 1, 58: 1, 5458: 1, 2608: 1, 5363: 1, 5456: 1, 220: 1, 72: 1, 5348: 1, 3078: 1, 5528: 1, 5704: 1, 5434: 1, 5752: 1, 2927: 1, 2597: 1, 5522: 1, 5703: 1, 5866: 1, 2620: 1, 91: 1, 6759: 1, 1595: 1, 5435: 1, 5736: 1, 81: 1, 1607: 1, 2938: 1, 165: 1, 522: 1, 37: 1, 5914: 1, 5738: 1, 6: 1, 2577: 1, 5917: 1, 2596: 1, 5910: 1, 76: 1, 5763: 1, 1198: 1, 8: 1, 520: 1, 519: 1})


In [31]:
text[1199]

'43 But know this, that if the goodman of the house had known in what watch the thief would come, he would have watched, and would not have suffered his house to be broken up.'

In [24]:
for t in scoreTuples[0:1000]:
    if counts[t[0]]>15:
#         print('skipped')
        continue
    else:
        print('Original Sentence: '+text[t[0]])
        print('Similar Sentence is from: '+t[1])
        print('Syntactic Score: ',t[3])
        print('Semantic Score: ',t[4])
        print(reducedBooks[t[1]][t[2]])
        print('\n\n')

Original Sentence: 20 But while he thought on these things, behold, the angel of the LORD appeared unto him in a dream, saying, Joseph, thou son of David, fear not to take unto thee Mary thy wife.
Similar Sentence is from: genesis
Syntactic Score:  0.922024047174
Semantic Score:  0.916171371937
1 And the LORD said unto Noah, Come thou and all thy house into the
ark; for thee have I seen righteous before me in this generation.



Original Sentence: 40 And the King shall answer and say unto them, Verily I say unto you, Inasmuch as ye have done it unto one of the least of these my brethren, ye have done it unto me.
Similar Sentence is from: samuel-2
Syntactic Score:  0.908029911541
Semantic Score:  0.919179081917
12 Go and say unto
David, Thus saith the LORD, I offer thee three things; choose thee one
of them, that I may do it unto thee.



Original Sentence: 31 When the Son of man shall come in his glory, and all the holy angels with him, then shall he sit upon the throne of his glory.
S

Syntactic Score:  0.82747708837
Semantic Score:  0.818390548229
13 But yet in it shall be a tenth, and it shall return, and shall be
eaten.



Original Sentence: 9 And now also the axe is laid unto the root of the trees.
Similar Sentence is from: leviticus
Syntactic Score:  0.859038638525
Semantic Score:  0.786710500717
3 And he shall offer of the sacrifice of the peace offering an
offering made by fire unto the LORD; the fat that covereth the
inwards, and all the fat that is upon the inwards, 3.



Original Sentence: 9 And now also the axe is laid unto the root of the trees.
Similar Sentence is from: genesis
Syntactic Score:  0.834541292776
Semantic Score:  0.81103682518
17 And God said unto Noah, This is the token of the covenant, which
I have established between me and all flesh that is upon the earth.



Original Sentence: 41 Two women shall be grinding at the mill; the one shall be taken, and the other left.
Similar Sentence is from: isaiah
Syntactic Score:  0.825714453729
Semanti

In [32]:
text

['The New Testament of the King James Bible     The Gospel According to Saint Matthew   1.',
 '1 The book of the generation of Jesus Christ, the son of David, the son of Abraham.',
 '2 Abraham begat Isaac; and Isaac begat Jacob; and Jacob begat Judas and his brethren; 1.',
 '3 And Judas begat Phares and Zara of Thamar; and Phares begat Esrom; and Esrom begat Aram; 1.',
 '4 And Aram begat Aminadab; and Aminadab begat Naasson; and Naasson begat Salmon; 1.',
 '5 And Salmon begat Booz of Rachab; and Booz begat Obed of Ruth; and Obed begat Jesse; 1.',
 '6 And Jesse begat David the king; and David the king begat Solomon of her that had been the wife of Urias; 1.',
 '7 And Solomon begat Roboam; and Roboam begat Abia; and Abia begat Asa; 1.',
 '8 And Asa begat Josaphat; and Josaphat begat Joram; and Joram begat Ozias; 1.',
 '9 And Ozias begat Joatham; and Joatham begat Achaz; and Achaz begat Ezekias; 1.',
 '10 And Ezekias begat Manasses; and Manasses begat Amon; and Amon begat Josias; 1.',
 '1

In [33]:
text.index('23 Behold, a virgin shall be with child, and shall bring forth a son, and they shall call his name Emmanuel, which being interpreted is, God with us.')

26

In [46]:
s=list()
for t in scoreTuples:
    if t[0]==26:
        s.append(t)

In [47]:
len(s)

19757

In [48]:
s.sort(key=lambda tup: tup[5],reverse=True)

In [49]:
len(s)

19757

In [45]:
for t in scoreTuples[0:10]:
    print('Original Sentence: '+text[t[0]])
    print('Similar Sentence is from: '+t[1])
    print('Syntactic Score: ',t[3])
    print('Semantic Score: ',t[4])
    print('Average: ',t[5])
    print(reducedBooks[t[1]][t[2]])
    print('\n\n')

Original Sentence: 20 But while he thought on these things, behold, the angel of the LORD appeared unto him in a dream, saying, Joseph, thou son of David, fear not to take unto thee Mary thy wife.
Similar Sentence is from: genesis
Syntactic Score:  0.922024047174
Semantic Score:  0.916171371937
Average:  0.919097709555
1 And the LORD said unto Noah, Come thou and all thy house into the
ark; for thee have I seen righteous before me in this generation.



Original Sentence: 40 And the King shall answer and say unto them, Verily I say unto you, Inasmuch as ye have done it unto one of the least of these my brethren, ye have done it unto me.
Similar Sentence is from: samuel-2
Syntactic Score:  0.908029911541
Semantic Score:  0.919179081917
Average:  0.913604496729
12 Go and say unto
David, Thus saith the LORD, I offer thee three things; choose thee one
of them, that I may do it unto thee.



Original Sentence: 31 When the Son of man shall come in his glory, and all the holy angels with him,

In [50]:
for t in s[0:10]:
    print('Original Sentence: '+text[t[0]])
    print('Similar Sentence is from: '+t[1])
    print('Syntactic Score: ',t[3])
    print('Semantic Score: ',t[4])
    print('Average: ',t[5])
    print(reducedBooks[t[1]][t[2]])
    print('\n\n')

Original Sentence: 23 Behold, a virgin shall be with child, and shall bring forth a son, and they shall call his name Emmanuel, which being interpreted is, God with us.
Similar Sentence is from: isaiah
Syntactic Score:  0.911016855099
Semantic Score:  0.88051789999
Average:  0.895767377544
14 Therefore
the Lord himself shall give you a sign; Behold, a virgin shall
conceive, and bear a son, and shall call his name Immanuel.



Original Sentence: 23 Behold, a virgin shall be with child, and shall bring forth a son, and they shall call his name Emmanuel, which being interpreted is, God with us.
Similar Sentence is from: Deuteronomy
Syntactic Score:  0.888943433711
Semantic Score:  0.858178138733
Average:  0.873560786222
11 And it shall be, if it make thee answer of peace, and open unto
thee, then it shall be, that all the people that is found therein
shall be tributaries unto thee, and they shall serve thee.



Original Sentence: 23 Behold, a virgin shall be with child, and shall bring fo

In [54]:
for i in range(10):
    print(scoreTuples[i])

(0, 'isaiah', 0, 0.73661143664473461, 0.66873633861541748, 0.70267388763007599)
(0, 'chronicle-2', 175, 0.78160756536761211, 0.58338910341262817, 0.68249833439012009)
(0, 'Joshua', 0, 0.74713248024704959, 0.61433702707290649, 0.68073475365997804)
(0, 'joel', 1, 0.80218035387221076, 0.54946786165237427, 0.67582410776229251)
(0, 'Proverbs', 208, 0.79363052888424268, 0.55176526308059692, 0.6726978959824198)
(0, 'daniel', 0, 0.74713248024704959, 0.59763050079345703, 0.67238149052025331)
(0, 'Proverbs', 1, 0.78845700989204792, 0.55149662494659424, 0.66997681741932102)
(0, 'nehemiah', 0, 0.74713248024704959, 0.5919157862663269, 0.66952413325668825)
(0, 'genesis', 829, 0.76483778652239021, 0.56255632638931274, 0.66369705645585153)
(0, 'end', 563, 0.83233872546007415, 0.48736527562141418, 0.65985200054074422)


In [211]:
potential="../data/potential/"
booksList=os.listdir(potential)

In [212]:
totalPotentialSentences=0
for bk in booksList:
    totalPotentialSentences=totalPotentialSentences+len(reducedBooks[bk])

In [213]:
totalPotentialSentences

550

In [216]:
finalTuples=list()
k=0
i=0
while i<len(scoreTuples):
    print(k)
    senttups=scoreTuples[i:i+totalPotentialSentences]
    senttups.sort(key=lambda tup: tup[5],reverse=True)
    if senttups[0][5]>0.86:
        finalTuples.append(senttups[0])
#     for j in range(len(senttups)):
#         if senttups[j][5]>0.85:
#             finalTuples.append(senttups[j])
#         else:
#             break
    i=i+totalPotentialSentences
    k=k+1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2176
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189
2190
2191
2192
2193
2194
2195
2196
2197
2198
2199
2200
2201
2202
2203
2204
2205
2206
2207
2208
2209
2210
2211
2212
2213
2214
2215
2216
2217
2218
2219
2220
2221
2222
2223
2224
2225
2226
2227
2228
2229
2230
2231
2232
2233
2234
2235
2236
2237
2238
2239
2240
2241
2242
2243
2244
2245
2246
2247
2248
2249
2250
2251
2252
2253
2254
2255
2256
2257
2258
2259
2260
2261
2262
2263
2264
2265
2266
2267


3743
3744
3745
3746
3747
3748
3749
3750
3751
3752
3753
3754
3755
3756
3757
3758
3759
3760
3761
3762
3763
3764
3765
3766
3767
3768
3769
3770
3771
3772
3773
3774
3775
3776
3777
3778
3779
3780
3781
3782
3783
3784
3785
3786
3787
3788
3789
3790
3791
3792
3793
3794
3795
3796
3797
3798
3799
3800
3801
3802
3803
3804
3805
3806
3807
3808
3809
3810
3811
3812
3813
3814
3815
3816
3817
3818
3819
3820
3821
3822
3823
3824
3825
3826
3827
3828
3829
3830
3831
3832
3833
3834
3835
3836
3837
3838
3839
3840
3841
3842
3843
3844
3845
3846
3847
3848
3849
3850
3851
3852
3853
3854
3855
3856
3857
3858
3859
3860
3861
3862
3863
3864
3865
3866
3867
3868
3869
3870
3871
3872
3873
3874
3875
3876
3877
3878
3879
3880
3881
3882
3883
3884
3885
3886
3887
3888
3889
3890
3891
3892
3893
3894
3895
3896
3897
3898
3899
3900
3901
3902
3903
3904
3905
3906
3907
3908
3909
3910
3911
3912
3913
3914
3915
3916
3917
3918
3919
3920
3921
3922
3923
3924
3925
3926
3927
3928
3929
3930
3931
3932
3933
3934
3935
3936
3937
3938
3939
3940
3941
3942


5422
5423
5424
5425
5426
5427
5428
5429
5430
5431
5432
5433
5434
5435
5436
5437
5438
5439
5440
5441
5442
5443
5444
5445
5446
5447
5448
5449
5450
5451
5452
5453
5454
5455
5456
5457
5458
5459
5460
5461
5462
5463
5464
5465
5466
5467
5468
5469
5470
5471
5472
5473
5474
5475
5476
5477
5478
5479
5480
5481
5482
5483
5484
5485
5486
5487
5488
5489
5490
5491
5492
5493
5494
5495
5496
5497
5498
5499
5500
5501
5502
5503
5504
5505
5506
5507
5508
5509
5510
5511
5512
5513
5514
5515
5516
5517
5518
5519
5520
5521
5522
5523
5524
5525
5526
5527
5528
5529
5530
5531
5532
5533
5534
5535
5536
5537
5538
5539
5540
5541
5542
5543
5544
5545
5546
5547
5548
5549
5550
5551
5552
5553
5554
5555
5556
5557
5558
5559
5560
5561
5562
5563
5564
5565
5566
5567
5568
5569
5570
5571
5572
5573
5574
5575
5576
5577
5578
5579
5580
5581
5582
5583
5584
5585
5586
5587
5588
5589
5590
5591
5592
5593
5594
5595
5596
5597
5598
5599
5600
5601
5602
5603
5604
5605
5606
5607
5608
5609
5610
5611
5612
5613
5614
5615
5616
5617
5618
5619
5620
5621


7086
7087
7088
7089
7090
7091
7092
7093
7094
7095
7096
7097
7098
7099
7100
7101
7102
7103
7104
7105
7106
7107
7108
7109
7110
7111
7112
7113
7114
7115
7116
7117
7118
7119
7120
7121
7122
7123
7124
7125
7126
7127
7128
7129
7130
7131
7132
7133
7134
7135
7136
7137
7138
7139
7140
7141
7142
7143
7144
7145
7146
7147
7148
7149
7150
7151
7152
7153
7154
7155
7156
7157
7158
7159
7160
7161
7162
7163
7164
7165
7166
7167
7168
7169
7170
7171
7172
7173
7174
7175
7176
7177
7178
7179
7180
7181
7182
7183
7184
7185
7186
7187
7188
7189
7190
7191
7192
7193
7194
7195
7196
7197
7198
7199
7200
7201
7202
7203
7204
7205
7206
7207
7208
7209
7210
7211
7212
7213
7214
7215
7216
7217
7218
7219
7220
7221
7222
7223
7224
7225
7226
7227
7228
7229
7230
7231
7232
7233
7234
7235
7236
7237
7238
7239
7240
7241
7242
7243
7244
7245
7246
7247
7248
7249
7250
7251
7252
7253
7254
7255
7256
7257
7258
7259
7260
7261
7262
7263
7264
7265
7266
7267
7268
7269
7270
7271
7272
7273
7274
7275
7276
7277
7278
7279
7280
7281
7282
7283
7284
7285


8821
8822
8823
8824
8825
8826
8827
8828
8829
8830
8831
8832
8833
8834
8835
8836
8837
8838
8839
8840
8841
8842
8843
8844
8845
8846
8847
8848
8849
8850
8851
8852
8853
8854
8855
8856
8857
8858
8859
8860
8861
8862
8863
8864
8865
8866
8867
8868
8869
8870
8871
8872
8873
8874
8875
8876
8877
8878
8879
8880
8881
8882
8883
8884
8885
8886
8887
8888
8889
8890
8891
8892
8893
8894
8895
8896
8897
8898
8899
8900
8901
8902
8903
8904
8905
8906
8907
8908
8909
8910
8911
8912
8913
8914
8915
8916
8917
8918
8919
8920
8921
8922
8923
8924
8925
8926
8927
8928
8929
8930
8931
8932
8933
8934
8935
8936
8937
8938
8939
8940
8941
8942
8943
8944
8945
8946
8947
8948
8949
8950
8951
8952
8953
8954
8955
8956
8957
8958
8959
8960
8961
8962
8963
8964
8965
8966
8967
8968
8969
8970
8971
8972
8973
8974
8975
8976
8977
8978
8979
8980
8981
8982
8983
8984
8985
8986
8987
8988
8989
8990
8991
8992
8993
8994
8995
8996
8997
8998
8999
9000
9001
9002
9003
9004
9005
9006
9007
9008
9009
9010
9011
9012
9013
9014
9015
9016
9017
9018
9019
9020


10500
10501
10502
10503
10504
10505
10506
10507
10508
10509
10510
10511
10512
10513
10514
10515
10516
10517
10518
10519
10520
10521
10522
10523
10524
10525
10526
10527
10528
10529
10530
10531
10532
10533
10534
10535
10536
10537
10538
10539
10540
10541
10542
10543
10544
10545
10546
10547
10548
10549
10550
10551
10552
10553
10554
10555
10556
10557
10558
10559
10560
10561
10562
10563
10564
10565
10566
10567
10568
10569
10570
10571
10572
10573
10574
10575
10576
10577
10578
10579
10580
10581
10582
10583
10584
10585
10586
10587
10588
10589
10590
10591
10592
10593
10594
10595
10596
10597
10598
10599
10600
10601
10602
10603
10604
10605
10606
10607
10608
10609
10610
10611
10612
10613
10614
10615
10616
10617
10618
10619
10620
10621
10622
10623
10624
10625
10626
10627
10628
10629
10630
10631
10632
10633
10634
10635
10636
10637
10638
10639
10640
10641
10642
10643
10644
10645
10646
10647
10648
10649
10650
10651
10652
10653
10654
10655
10656
10657
10658
10659
10660
10661
10662
10663
10664
10665
1066

In [217]:
len(finalTuples)

466

In [218]:
finalTuples.sort(key=lambda tup: tup[5],reverse=True)

In [219]:
def jacardNouns(sent1,sent2):
    nouns1=[]
    for word,pos in nltk.pos_tag(word_tokenize(sent1)):
        if pos.startswith('NN'):
            nouns1.append(word)
    nouns2=[]
    for word,pos in nltk.pos_tag(word_tokenize(sent2)):
        if pos.startswith('NN'):
            nouns2.append(word)
#     print(nouns1)
#     print(nouns2)
    if len(set(nouns1).union(nouns2))==0:
        ratio=0
    else:
        ratio = len(set(nouns1).intersection(nouns2)) / float(len(set(nouns1).union(nouns2)))        
    return ratio

In [220]:
finalTuples=finalTuples[0:100]

In [221]:
from nltk.tokenize import word_tokenize

In [222]:
newTuples=list()
for tup in finalTuples:
    originalSent=text[tup[0]]
    refSent=reducedBooks[tup[1]][tup[2]]
    nounScore=jacardNouns(originalSent,refSent)
#     s1,s2=jacardNouns(originalSent,refSent)
#     nounScore=sentenceSimilarity(s1,s2)
    newTuples.append(tup+(nounScore,))

In [223]:
len(newTuples)

100

In [224]:
from operator import itemgetter

In [225]:
newTuples.sort(key=itemgetter(6,5),reverse=True)

In [228]:
f=open('../output/testPackage/outputScores-further-filtered-nounSorted.txt','a')

In [229]:
lines=list()
i=1
for tup in newTuples:
    j=str(i)
    lines.append(j)
    lines.append('\n')
    lines.append('New Sentence: ')
    lines.append(text[tup[0]])
    lines.append('\n')
    lines.append('Reference Sentence: ')
    lines.append(reducedBooks[tup[1]][tup[2]])
    lines.append('\n')
    lines.append('Syntactic Similarity: ')
    lines.append(str(tup[3]))
    lines.append('\n')
    lines.append('Semantic Similarity: ')
    lines.append(str(tup[4]))
    lines.append('\n\n')
    i=i+1
f.writelines(lines)

In [127]:
books=dict()
for file in booksList:
    print(file)
    candidate=open(potential+file)
    rawtext=candidate.read()
    candidate.close()
    rawtext = strip_headers(rawtext).strip()
    candidate=rawtext.replace('\n',' ')
    candidate=rawtext.replace(':','. ')
    candidate=sent_tokenize(candidate)
    candidate = list(filter(lambda x: len(x)>5, candidate))
    books[file]=candidate

Joshua
hosea
exodus
esther
amos
Ezra
genesis
kings-3
leviticus
samuel-2
Proverbs
Deuteronomy
end
isaiah
chronicle-2
daniel
joel
Numbers
nehemiah
jeremiah-2
Psalms
jeremiah-1
Judges
samuel-1
chronicle-1
preacher
Ruth
kings-4
job


In [128]:
len(books)

29

In [130]:
finalOutput=list()
for tup in newTuples:
    output=list()
    
    originalSent=tup[0]
    refBook=tup[1]
    refSentReduced=tup[2]
    ref=reducedBooks[refBook][refSentReduced]
    refSent=books[refBook].index(ref)
    bk=books[refBook]
    
    if originalSent==0:
        original=text[originalSent:originalSent+2]
    if originalSent==len(text)-1:
        original=text[originalSent-2:originalSent+1]
    else:
        original=text[originalSent-1:originalSent+2]
    
    
    if refSent==0:
        reference=bk[refSent:refSent+2]
    if refSent==len(bk)-1:
        reference=bk[refSent-2:refSent+1]
    else:
        reference=bk[refSent-1:refSent+2]

    original=' '.join(original)
    reference=' '.join(reference)
    
    finalOutput.append((original,reference))
        
        

In [145]:
f=open('../output/annotated/outputScores-further-filtered-nounSorted.txt','w')

In [146]:
lines=list()
i=0
for output in finalOutput:
    j=str(i+1)
    lines.append(j)
    lines.append('\n')
    lines.append('New Sentence: ')
    lines.append(output[0])
    lines.append('\n')
    lines.append('Reference Sentence: ')
    lines.append(output[1])
    lines.append('\n\n')
    lines.append('\n')
    lines.append('Syntactic Similarity: ')
    lines.append(str(newTuples[i][3]))
    lines.append('\n')
    lines.append('Semantic Similarity: ')
    lines.append(str(newTuples[i][4]))
    lines.append('\n\n')
    i=i+1
f.writelines(lines)

In [147]:
noyes22=[5,28,24,21,36,40,25,27,35]

In [182]:
f=open('../output/annotated/2-2-further-filtered-nounSorted.txt','a')

In [183]:
lines=list()
i=1
for output in finalOutput:
    if i not in noyes22:
        i=i+1
        continue
    j=str(i)
    lines.append(j)
    lines.append('\n')
    lines.append('New Sentence: ')
    lines.append(output[0])
    lines.append('\n')
    lines.append('Reference Sentence: ')
    lines.append(output[1])
    lines.append('\n\n')
    lines.append('\n')
    lines.append('Syntactic Similarity: ')
    lines.append(str(newTuples[i][3]))
    lines.append('\n')
    lines.append('Semantic Similarity: ')
    lines.append(str(newTuples[i][4]))
    lines.append('\n\n')
    i=i+1
f.writelines(lines)

In [184]:
len(lines)

135

In [185]:
noyes13=[8,3,15,18,37,22]

In [188]:
f=open('../output/annotated/1-3-further-filtered-nounSorted.txt','a')

In [189]:
lines=list()
i=1
for output in finalOutput:
    if i not in noyes13:
        i=i+1
        continue
    j=str(i)
    lines.append(j)
    lines.append('\n')
    lines.append('New Sentence: ')
    lines.append(output[0])
    lines.append('\n')
    lines.append('Reference Sentence: ')
    lines.append(output[1])
    lines.append('\n\n')
    lines.append('\n')
    lines.append('Syntactic Similarity: ')
    lines.append(str(newTuples[i][3]))
    lines.append('\n')
    lines.append('Semantic Similarity: ')
    lines.append(str(newTuples[i][4]))
    lines.append('\n\n')
    i=i+1
f.writelines(lines)

In [190]:
noyes31=[31,86,81,29,75,4]

In [193]:
f=open('../output/annotated/3-1-further-filtered-nounSorted.txt','a')

In [194]:
lines=list()
i=1
for output in finalOutput:
    if i not in noyes31:
        i=i+1
        continue
    j=str(i)
    lines.append(j)
    lines.append('\n')
    lines.append('New Sentence: ')
    lines.append(output[0])
    lines.append('\n')
    lines.append('Reference Sentence: ')
    lines.append(output[1])
    lines.append('\n\n')
    lines.append('\n')
    lines.append('Syntactic Similarity: ')
    lines.append(str(newTuples[i][3]))
    lines.append('\n')
    lines.append('Semantic Similarity: ')
    lines.append(str(newTuples[i][4]))
    lines.append('\n\n')
    i=i+1
f.writelines(lines)

In [195]:
def getTree(sent): 
    parsed=getNLPToks(sent)
    x=parsed['parse']
    s=''
    for i in x:
        s=s+i
    y=Tree.fromstring(s)
    return y

In [196]:
def getNLPToks(rawSentence):
    output = nlp.annotate(rawSentence, properties={'annotators': 'tokenize,ssplit,pos,parse','outputFormat': 'json','timeout':'50000'})
    output=ast.literal_eval(output)
    tokens = output['sentences'][0]['tokens']
    parse = output['sentences'][0]['parse'].split("\n")
    return {
        'toks':tokens, 'parse':parse
    }

In [197]:
from nltk.draw.tree import TreeView

In [202]:
lines=list()
i=1
for tup in newTuples:
    if i not in noyes22:
        i=i+1
        continue
    j=str(i)
    original=text[tup[0]]
    ref=reducedBooks[tup[1]][tup[2]]
    tr1=getTree(original)
    tr2=getTree(ref)
    TreeView(tr1)._cframe.print_to_file('../output/annotated/2-2/'+j+'-original.ps')
    TreeView(tr2)._cframe.print_to_file('../output/annotated/2-2/'+j+'-reference.ps')
    i=i+1
f.writelines(lines)

In [203]:
lines=list()
i=1
for tup in newTuples:
    if i not in noyes13:
        i=i+1
        continue
    j=str(i)
    original=text[tup[0]]
    ref=reducedBooks[tup[1]][tup[2]]
    tr1=getTree(original)
    tr2=getTree(ref)
    TreeView(tr1)._cframe.print_to_file('../output/annotated/1-3/'+j+'-original.ps')
    TreeView(tr2)._cframe.print_to_file('../output/annotated/1-3/'+j+'-reference.ps')
    i=i+1
f.writelines(lines)

In [204]:
lines=list()
i=1
for tup in newTuples:
    if i not in noyes31:
        i=i+1
        continue
    j=str(i)
    original=text[tup[0]]
    ref=reducedBooks[tup[1]][tup[2]]
    tr1=getTree(original)
    tr2=getTree(ref)
    TreeView(tr1)._cframe.print_to_file('../output/annotated/3-1/'+j+'-original.ps')
    TreeView(tr2)._cframe.print_to_file('../output/annotated/3-1/'+j+'-reference.ps')
    i=i+1
f.writelines(lines)

Exploring further syntactic and semantic similarity using parse trees

In [2]:
def tree(): 
    return defaultdict(tree)


def _leadingSpaces_(target):
    return len(target) - len(target.lstrip())

def _findParent_(curIndent, parid, treeRef):
    tmpid = parid
    while (curIndent <= treeRef[tmpid]['indent']):
        tmpid = treeRef[tmpid]['parid']
    return tmpid


def generateTree(rawTokens, treeRef):

    # (token
    REGEX_OPEN = r"^\s*\(([a-zA-Z0-9_']*)\s*$"
    # (token (tok1 tok2) (tok3 tok4) .... (tokx toky))
    REGEX_COMP = r"^\s*\(([a-zA-Z0-9_']+)\s*((?:[(]([a-zA-Z0-9_;.,?'!]+)\s*([a-zA-Z0-9_;\.,?!']+)[)]\s*)+)"    
    # (, ,) as stand-alone. Used for match() not search()
    REGEX_PUNC = r"^\s*\([,!?.'\"]\s*[,!?.'\"]\)"
    # (tok1 tok2) as stand-alone
    REGEX_SOLO_PAIR = r"^\s*\(([a-zA-Z0-9_']+)\s*([a-zA-Z0-9_']+)\)"
    # (tok1 tok2) used in search()
    REGEX_ISOL_IN_COMP = r"\(([a-zA-Z0-9_;.,?!']+)\s*([a-zA-Z0-9_;.,?!']+)\)"
    # (punc punc) used in search()
    REGEX_PUNC_SOLO = r"\([,!?.'\"]\s*[,!?.'\"]\)"
   
    treeRef[len(treeRef)] = {'curid':0, 
                             'parid':-1, 
                             'posOrTok':'ROOT', 
                             'indent':0,
                            'children':[],
                            'childrenTok':[]}
    ID_CTR = 1
    
    for tok in rawTokens[1:]:
        
        curIndent = _leadingSpaces_(tok) 
        parid = _findParent_(curIndent, ID_CTR-1, treeRef)
        
        # CHECK FOR COMPOSITE TOKENS
        checkChild = re.match(REGEX_COMP, tok)
        if (checkChild):
            treeRef[ID_CTR] = {'curid':ID_CTR, 
                               'parid':parid, 
                               'posOrTok':checkChild.group(1), 
                               'indent':curIndent,
                              'children':[],
                              'childrenTok':[]}
            upCTR = ID_CTR
            ID_CTR += 1
            
            subCheck = re.sub(REGEX_PUNC_SOLO,'',checkChild.group(2))
            subs = re.findall(REGEX_ISOL_IN_COMP, subCheck) 
            for ch in subs:
                treeRef[ID_CTR] = {'curid':ID_CTR, 
                                   'parid':upCTR, 
                                   'posOrTok':ch[0], 
                                   'indent':curIndent+2,
                                  'children':[],
                                  'childrenTok':[]}
                ID_CTR += 1
                treeRef[ID_CTR] = {'curid':ID_CTR, 
                                   'parid':ID_CTR-1, 
                                   'posOrTok':ch[1], 
                                   'indent':curIndent+2,
                                  'children':[],
                                  'childrenTok':[]}
                ID_CTR += 1
            continue
           

            
        checkSingle = re.match(REGEX_SOLO_PAIR, tok)
        if (checkSingle):
            treeRef[ID_CTR] = {'curid':ID_CTR, 
                               'parid':parid, 
                               'posOrTok':checkSingle.group(1), 
                               'indent':curIndent+2,
                              'children':[],
                              'childrenTok':[]}
            ID_CTR += 1
            treeRef[ID_CTR] = {'curid':ID_CTR, 
                               'parid':ID_CTR-1, 
                               'posOrTok':checkSingle.group(2), 
                               'indent':curIndent+2,
                              'children':[],
                              'childrenTok':[]}
            ID_CTR += 1
            continue
        
        
        checkPunc = re.match(REGEX_PUNC, tok)
        if (checkPunc): # ignore punctuation
            continue

        checkMatch = re.match(REGEX_OPEN, tok)
        if (checkMatch):
            treeRef[ID_CTR] = {'curid':ID_CTR, 
                               'parid':parid, 
                               'posOrTok':checkMatch.group(1), 
                               'indent':curIndent,
                              'children':[],
                              'childrenTok':[]}
            ID_CTR += 1
            continue

    return
            

def flipTree(treeRef):
    # Pass 1 fill in children
    for k,v in treeRef.items():
        if (k > 0):
            bisect.insort(treeRef[v['parid']]['children'], k)
    # Pass 2 map children to tokens
    for k,v in treeRef.items():
        if (k > 0):
            treeRef[k]['childrenTok'] = [treeRef[ch]['posOrTok'] for ch in treeRef[k]['children']]
    treeRef[0]['childrenTok'] = treeRef[1]['posOrTok']


In [3]:
def getNLPToks(rawSentence):
    output = nlp.annotate(rawSentence, properties={'annotators': 'tokenize,ssplit,pos,parse','outputFormat': 'json','timeout':'50000'})
    output=ast.literal_eval(output)
    tokens = output['sentences'][0]['tokens']
    parse = output['sentences'][0]['parse'].split("\n")
    return {
        'toks':tokens, 'parse':parse
    }

In [4]:
public='/home/users2/mehrotsh/scripts/packages/stanford-corenlp-full-2018-02-27/'
personal='/home/samarth/stanford-corenlp-full-2018-02-27/'
nlp = StanfordCoreNLP(public)

In [5]:
def getTree(sent): 
    parsed=getNLPToks(sent)
    x=parsed['parse']
    s=''
    for i in x:
        s=s+i
    y=Tree.fromstring(s)
    return y

In [6]:
tr=getTree('Cats are men and not green monkeys')

In [13]:
tr2=getTree('Cates are men and not green monkeys')

In [8]:
print(tr.flatten())

(ROOT Cats are men and not green monkeys)


In [25]:
subt=tr.subtrees()
subt2=tr2.subtrees()
t=[]
for i in subt:
    print(i.pprint())


(ROOT
  (S
    (NP (NNS Cats))
    (VP
      (VBP are)
      (NP
        (NP (NNS men))
        (CC and)
        (RB not)
        (NP (JJ green) (NNS monkeys))))))
None
(S
  (NP (NNS Cats))
  (VP
    (VBP are)
    (NP
      (NP (NNS men))
      (CC and)
      (RB not)
      (NP (JJ green) (NNS monkeys)))))
None
(NP (NNS Cats))
None
(NNS Cats)
None
(VP
  (VBP are)
  (NP (NP (NNS men)) (CC and) (RB not) (NP (JJ green) (NNS monkeys))))
None
(VBP are)
None
(NP (NP (NNS men)) (CC and) (RB not) (NP (JJ green) (NNS monkeys)))
None
(NP (NNS men))
None
(NNS men)
None
(CC and)
None
(RB not)
None
(NP (JJ green) (NNS monkeys))
None
(JJ green)
None
(NNS monkeys)
None


In [47]:
tr.pformat(nodesep='',quotes=False)

'(ROOT\n  (S\n    (NP (NNS Cats))\n    (VP\n      (VBP are)\n      (NP\n        (NP (NNS men))\n        (CC and)\n        (RB not)\n        (NP (JJ green) (NNS monkeys))))))'

In [29]:
s=getNLPToks('Cats are men and not green monkeys')

In [40]:
s['parse']

['(ROOT',
 '  (S',
 '    (NP (NNS Cats))',
 '    (VP (VBP are)',
 '      (NP',
 '        (NP (NNS men))',
 '        (CC and)',
 '        (RB not)',
 '        (NP (JJ green) (NNS monkeys))))))']

In [33]:
sentence=list()
sentence.append(tr.pformat())

In [34]:
sentence

['(ROOT\n  (S\n    (NP (NNS Cats))\n    (VP\n      (VBP are)\n      (NP\n        (NP (NNS men))\n        (CC and)\n        (RB not)\n        (NP (JJ green) (NNS monkeys))))))']

In [95]:
for tr in t:
    print(tr.leaves())

['Cats']
['men', 'and', 'not', 'green', 'monkeys']
['men']
['green', 'monkeys']


In [96]:
allNouns=list()
for tr in t:
    for noun in tr.leaves():
        allNouns.append(noun)

In [97]:
allNouns=list(set(allNouns))

In [98]:
allNouns

['monkeys', 'men', 'Cats', 'not', 'green', 'and']

In [23]:
tempTree=tree()

In [24]:
parsed=getNLPToks('cats are men')

In [25]:
parsed['parse']

['(ROOT',
 '  (S',
 '    (NP (NNS cats))',
 '    (VP (VBP are)',
 '      (NP (NNS men)))))']

In [30]:
from nltk.tokenize import word_tokenize


In [35]:
text = word_tokenize("They refuse to permit us to obtain the refuse permit")
pos=nltk.pos_tag(text)

In [36]:
grammar = "NP: {<DT>?<JJ>*<NN>}"

In [37]:
cp = nltk.RegexpParser(grammar)

In [38]:
result = cp.parse(pos)

In [48]:
print(result)

(S
  They/PRP
  refuse/VBP
  to/TO
  permit/VB
  us/PRP
  to/TO
  obtain/VB
  (NP the/DT refuse/NN)
  (NP permit/NN))


In [49]:
result.pprint()

(S
  They/PRP
  refuse/VBP
  to/TO
  permit/VB
  us/PRP
  to/TO
  obtain/VB
  (NP the/DT refuse/NN)
  (NP permit/NN))


In [40]:
from textblob import TextBlob

In [45]:
blob=TextBlob('the quick brown fox jumped over the lazy dog')

In [46]:
blob.noun_phrases

WordList(['quick brown fox', 'lazy dog'])

In [47]:
for word in blob.noun_phrases:
    print(word)

quick brown fox
lazy dog


In [103]:
sent1='the quick brown fox jumped over the lazy dog'

In [104]:
nouns=[]
for word,pos in nltk.pos_tag(word_tokenize(sent1)):
    if pos.startswith('NN'):
        nouns.append(word)

In [105]:
nouns

['brown', 'fox', 'dog']

In [61]:
pickle_off = open("../../backup/output/bible-temp-2/scoreTuples.pickle","rb")
scoreTuples = pickle.load(pickle_off)

In [62]:
pickle_off = open("../../backup/output/bible-temp-2/reducedBooks.pickle","rb")
reducedBooks = pickle.load(pickle_off)

In [63]:
scoreTuples.sort(key=lambda tup: tup[0])

In [65]:
potential="../data/potential/"
booksList=os.listdir(potential)

In [66]:
totalPotentialSentences=0
for bk in booksList:
    totalPotentialSentences=totalPotentialSentences+len(reducedBooks[bk])

In [67]:
totalPotentialSentences

19757

In [68]:
finalTuples=list()
k=0
i=0
while i<len(scoreTuples):
    print(k)
    senttups=scoreTuples[i:i+19757]
    senttups.sort(key=lambda tup: tup[5],reverse=True)
    if senttups[0][5]>0.89:
        finalTuples.append(senttups[0])
#     for j in range(len(senttups)):
#         if senttups[j][5]>0.85:
#             finalTuples.append(senttups[j])
#         else:
#             break
    i=i+19757
    k=k+1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124


3752
3753
3754
3755
3756
3757
3758
3759
3760
3761
3762
3763
3764
3765
3766
3767
3768
3769
3770
3771
3772
3773
3774
3775
3776
3777
3778
3779
3780
3781
3782
3783
3784
3785
3786
3787
3788
3789
3790
3791
3792
3793
3794
3795
3796
3797
3798
3799
3800
3801
3802
3803
3804
3805
3806
3807
3808
3809
3810
3811
3812
3813
3814
3815
3816
3817
3818
3819
3820
3821
3822
3823
3824
3825
3826
3827
3828
3829
3830
3831
3832
3833
3834
3835
3836
3837
3838
3839
3840
3841
3842
3843
3844
3845
3846
3847
3848
3849
3850
3851
3852
3853
3854
3855
3856
3857
3858
3859
3860
3861
3862
3863
3864
3865
3866
3867
3868
3869
3870
3871
3872
3873
3874
3875
3876
3877
3878
3879
3880
3881
3882
3883
3884
3885
3886
3887
3888
3889
3890
3891
3892
3893
3894
3895
3896
3897
3898
3899
3900
3901
3902
3903
3904
3905
3906
3907
3908
3909
3910
3911
3912
3913
3914
3915
3916
3917
3918
3919
3920
3921
3922
3923
3924
3925
3926
3927
3928
3929
3930
3931
3932
3933
3934
3935
3936
3937
3938
3939
3940
3941
3942
3943
3944
3945
3946
3947
3948
3949
3950
3951


5435
5436
5437
5438
5439
5440
5441
5442
5443
5444
5445
5446
5447
5448
5449
5450
5451
5452
5453
5454
5455
5456
5457
5458
5459
5460
5461
5462
5463
5464
5465
5466
5467
5468
5469
5470
5471
5472
5473
5474
5475
5476
5477
5478
5479
5480
5481
5482
5483
5484
5485
5486
5487
5488
5489
5490
5491
5492
5493
5494
5495
5496
5497
5498
5499
5500
5501
5502
5503
5504
5505
5506
5507
5508
5509
5510
5511
5512
5513
5514
5515
5516
5517
5518
5519
5520
5521
5522
5523
5524
5525
5526
5527
5528
5529
5530
5531
5532
5533
5534
5535
5536
5537
5538
5539
5540
5541
5542
5543
5544
5545
5546
5547
5548
5549
5550
5551
5552
5553
5554
5555
5556
5557
5558
5559
5560
5561
5562
5563
5564
5565
5566
5567
5568
5569
5570
5571
5572
5573
5574
5575
5576
5577
5578
5579
5580
5581
5582
5583
5584
5585
5586
5587
5588
5589
5590
5591
5592
5593
5594
5595
5596
5597
5598
5599
5600
5601
5602
5603
5604
5605
5606
5607
5608
5609
5610
5611
5612
5613
5614
5615
5616
5617
5618
5619
5620
5621
5622
5623
5624
5625
5626
5627
5628
5629
5630
5631
5632
5633
5634


7087
7088
7089
7090
7091
7092
7093
7094
7095
7096
7097
7098
7099
7100
7101
7102
7103
7104
7105
7106
7107
7108
7109
7110
7111
7112
7113
7114
7115
7116
7117
7118
7119
7120
7121
7122
7123
7124
7125
7126
7127
7128
7129
7130
7131
7132
7133
7134
7135
7136
7137
7138
7139
7140
7141
7142
7143
7144
7145
7146
7147
7148
7149
7150
7151
7152
7153
7154
7155
7156
7157
7158
7159
7160
7161
7162
7163
7164
7165
7166
7167
7168
7169
7170
7171
7172
7173
7174
7175
7176
7177
7178
7179
7180
7181
7182
7183
7184
7185
7186
7187
7188
7189
7190
7191
7192
7193
7194
7195
7196
7197
7198
7199
7200
7201
7202
7203
7204
7205
7206
7207
7208
7209
7210
7211
7212
7213
7214
7215
7216
7217
7218
7219
7220
7221
7222
7223
7224
7225
7226
7227
7228
7229
7230
7231
7232
7233
7234
7235
7236
7237
7238
7239
7240
7241
7242
7243
7244
7245
7246
7247
7248
7249
7250
7251
7252
7253
7254
7255
7256
7257
7258
7259
7260
7261
7262
7263
7264
7265
7266
7267
7268
7269
7270
7271
7272
7273
7274
7275
7276
7277
7278
7279
7280
7281
7282
7283
7284
7285
7286


8738
8739
8740
8741
8742
8743
8744
8745
8746
8747
8748
8749
8750
8751
8752
8753
8754
8755
8756
8757
8758
8759
8760
8761
8762
8763
8764
8765
8766
8767
8768
8769
8770
8771
8772
8773
8774
8775
8776
8777
8778
8779
8780
8781
8782
8783
8784
8785
8786
8787
8788
8789
8790
8791
8792
8793
8794
8795
8796
8797
8798
8799
8800
8801
8802
8803
8804
8805
8806
8807
8808
8809
8810
8811
8812
8813
8814
8815
8816
8817
8818
8819
8820
8821
8822
8823
8824
8825
8826
8827
8828
8829
8830
8831
8832
8833
8834
8835
8836
8837
8838
8839
8840
8841
8842
8843
8844
8845
8846
8847
8848
8849
8850
8851
8852
8853
8854
8855
8856
8857
8858
8859
8860
8861
8862
8863
8864
8865
8866
8867
8868
8869
8870
8871
8872
8873
8874
8875
8876
8877
8878
8879
8880
8881
8882
8883
8884
8885
8886
8887
8888
8889
8890
8891
8892
8893
8894
8895
8896
8897
8898
8899
8900
8901
8902
8903
8904
8905
8906
8907
8908
8909
8910
8911
8912
8913
8914
8915
8916
8917
8918
8919
8920
8921
8922
8923
8924
8925
8926
8927
8928
8929
8930
8931
8932
8933
8934
8935
8936
8937


10388
10389
10390
10391
10392
10393
10394
10395
10396
10397
10398
10399
10400
10401
10402
10403
10404
10405
10406
10407
10408
10409
10410
10411
10412
10413
10414
10415
10416
10417
10418
10419
10420
10421
10422
10423
10424
10425
10426
10427
10428
10429
10430
10431
10432
10433
10434
10435
10436
10437
10438
10439
10440
10441
10442
10443
10444
10445
10446
10447
10448
10449
10450
10451
10452
10453
10454
10455
10456
10457
10458
10459
10460
10461
10462
10463
10464
10465
10466
10467
10468
10469
10470
10471
10472
10473
10474
10475
10476
10477
10478
10479
10480
10481
10482
10483
10484
10485
10486
10487
10488
10489
10490
10491
10492
10493
10494
10495
10496
10497
10498
10499
10500
10501
10502
10503
10504
10505
10506
10507
10508
10509
10510
10511
10512
10513
10514
10515
10516
10517
10518
10519
10520
10521
10522
10523
10524
10525
10526
10527
10528
10529
10530
10531
10532
10533
10534
10535
10536
10537
10538
10539
10540
10541
10542
10543
10544
10545
10546
10547
10548
10549
10550
10551
10552
10553
1055

In [69]:
finalTuples.sort(key=lambda tup: tup[5],reverse=True)

In [70]:
fTuples=finalTuples[0:100]

In [52]:
def jacardNouns(sent1,sent2):
    nouns1=[]
    for word,pos in nltk.pos_tag(word_tokenize(sent1)):
        if pos.startswith('NN'):
            nouns1.append(word)
    nouns2=[]
    for word,pos in nltk.pos_tag(word_tokenize(sent2)):
        if pos.startswith('NN'):
            nouns2.append(word)
#     print(nouns1)
#     print(nouns2)
    if len(set(nouns1).union(nouns2))==0:
        ratio=0
    else:
        ratio = len(set(nouns1).intersection(nouns2)) / float(len(set(nouns1).union(nouns2)))        
    return nouns1,nouns2

In [78]:
def sentenceSimilarity(sent1,sent2):
    
#     sent1words=word_tokenize(sent1)
#     sent2words=word_tokenize(sent2)
    sent1words=sent1
    sent2words=sent2
    
    sent1tags=pos_tag(sent1words)
    sent2tags=pos_tag(sent2words)
    
    
    sent1tags=[t[1] for t in sent1tags]
    sent2tags=[t[1] for t in sent2tags]
    
    sent1synset=[]
    for i in range(len(sent1words)):
        sent1synset.append(tagged_to_synset(sent1words[i],sent1tags[i]))
        
    
    sent2synset=[]
    for i in range(len(sent2words)):
        sent2synset.append(tagged_to_synset(sent2words[i],sent2tags[i]))
        
        
    sent1synset = [ss for ss in sent1synset if ss is not None]
    sent2synset = [ss for ss in sent2synset if ss is not None]
 
    score, count = 0.0, 0
 
    # For each word in the first sentence
    for synset in sent1synset:
        # Get the similarity value of the most similar word in the other sentence
        try:
            best_score = max([synset.path_similarity(ss) for ss in sent2synset])
        except (TypeError,ValueError) as e:
            best_score=None
        # Check that the similarity could have been computed
        if best_score is not None:
            score += best_score
            count += 1
    if count==0:
        return 0
    else:
        score /= count
    return score
 
        
        
        
        
        
    

In [79]:
newTuples=list()
for tup in fTuples:
    originalSent=text[tup[0]]
    refSent=reducedBooks[tup[1]][tup[2]]
#     nounScore=jacardNouns(originalSent,refSent)
    s1,s2=jacardNouns(originalSent,refSent)
    nounScore=sentenceSimilarity(s1,s2)
    newTuples.append(tup+(nounScore,))
    

In [80]:
len(newTuples)

100

In [81]:
from operator import itemgetter

In [82]:
newTuples.sort(key=itemgetter(6,5),reverse=True)

In [156]:
# newTuples.sort(key=lambda tup: tup[6],reverse=True)

In [85]:
f=open('../output/sentencePairs-nounWordNetFiltered.txt','w')

In [86]:
lines=list()
i=1
for tup in newTuples:
    j=str(i)
    lines.append(j)
    lines.append('\n')
    lines.append('New Sentence: ')
    lines.append(text[tup[0]])
    lines.append('\n')
    lines.append('Reference Sentence: ')
    lines.append(reducedBooks[tup[1]][tup[2]])
    lines.append('\n\n')
    i=i+1
f.writelines(lines)

In [53]:
s1='And they brought unto him a penny'
s2='And they brought him unto Moses'

In [55]:
s1,s2=jacardNouns(s1,s2)


In [60]:
sentenceSimilarity(s1,s2)

0.07142857142857142

inter annotator similairty

In [77]:
import pandas as pd

In [78]:
df=pd.read_csv('../output/annotated.csv',header=None)

In [79]:
df

,0,1,2,3
0,n,n,n,n
1,y,y,y,y
2,n,n,n,n
3,y,y,y,y
4,n,n,n,n
5,n,n,n,n
6,n,y,y,n
7,n,n,n,n
8,n,n,n,n
9,n,n,n,n


In [80]:
no=list()
yes=list()

In [81]:
for index,row in df.iterrows():
    n=0
    y=0
    if row[0]=='y':
        y=y+1
    if row[0]=='n':
        n=n+1
    if row[1]=='y':
        y=y+1
    if row[1]=='n':
        n=n+1
    if row[2]=='y':
        y=y+1
    if row[2]=='n':
        n=n+1
    if row[3]=='y':
        y=y+1
    if row[3]=='n':
        n=n+1
    if (y+n)!=4:
        print('error')
    else:
        no.append(n)
        yes.append(y)

In [82]:
no[6]

2

In [26]:
df[0:10]

,0,1,2,3
0,n,n,n,n
1,y,y,y,y
2,n,n,n,n
3,y,y,y,y
4,n,n,n,n
5,n,n,n,n
6,n,y,y,n
7,n,n,n,n
8,n,n,n,n
9,n,n,n,n


In [27]:
yes[0:10]

[0, 4, 0, 4, 0, 0, 2, 0, 0, 0]

In [28]:
no[0:10]

[4, 0, 4, 0, 4, 4, 2, 4, 4, 4]

In [29]:
df[44:47]

,0,1,2,3
44,n,y,n,n
45,y,y,y,y
46,y,n,y,n


In [9]:
janis=list(df[0])

In [10]:
nils=list(df[1])

In [12]:
axel=list(df[2])

In [13]:
samarth=list(df[3])

In [15]:
from sklearn.metrics import cohen_kappa_score

In [16]:
cohen_kappa_score(janis,nils)

0.63692946058091282

In [17]:
cohen_kappa_score(janis,axel)

0.68415937803692906

In [18]:
cohen_kappa_score(janis,samarth)

0.76970317297850566

In [19]:
cohen_kappa_score(nils,axel)

0.64722483537158992

In [20]:
cohen_kappa_score(nils,samarth)

0.73012757605495582

In [21]:
cohen_kappa_score(axel,samarth)

0.76668222118525431

In [83]:
tuples=list()

In [84]:
for i in range(len(no)):
    tuples.append((no[i],yes[i]))

In [85]:
len(tuples)

100

In [86]:
import collections

In [87]:
counter=collections.Counter(tuples)

In [88]:
print(counter)

Counter({(4, 0): 61, (0, 4): 18, (2, 2): 9, (3, 1): 6, (1, 3): 6})


In [103]:
indicesOf22 = [i+1 for i, x in enumerate(tuples) if x == (2,2)]

In [104]:
indicesOf22

[7, 11, 17, 24, 47, 48, 57, 90, 96]

In [105]:
indicesOf13 = [i+1 for i, x in enumerate(tuples) if x == (1,3)]

In [106]:
indicesOf31 = [i+1 for i, x in enumerate(tuples) if x == (3,1)]

In [107]:
indicesOf13

[50, 52, 62, 82, 84, 93]

In [108]:
indicesOf31

[21, 26, 45, 49, 73, 76]